In [1]:
import os

os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

In [2]:
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
import glob
from IPython.core.display_functions import clear_output

In [ ]:
import sys  
sys.path.insert(1, '../../../')

import generate_dataset
import experiment, autoencoders

### Parameters

In [4]:
num_endmembers = 5
spectral_size = 1000
image_size = 100
epochs = 10
ae_loss = autoencoders.SAD
model_replicates = 5

scenes = ['chessboard', 'gaussian', 'dirichlet']

data_seeds = [1, 2, 3, 4, 5]
experiment_seed = 42

data_folder = 'generated data'
results_folder = 'results'

In [5]:
def display_results(metrics):
    results_order = ['PCA', 'N-FINDR + FCLS', 'VCA + FCLS', 'Dense AE', 'Convolutional AE', 'Transformer AE', 'Convolutional Transformer AE']

    metrics['SAD'] = metrics[['SAD_mean', 'SAD_std']].apply(lambda x: ' ± '.join(x.round(3).astype(str)), axis=1)
    metrics['PCC'] = metrics[['PCC_mean', 'PCC_std']].apply(lambda x: ' ± '.join(x.round(3).astype(str)), axis=1)
    metrics['MSE'] = metrics[['MSE_mean', 'MSE_std']].apply(lambda x: ' ± '.join(x.round(3).astype(str)), axis=1)

    display(metrics.set_index('index').reindex(results_order)[['SAD', 'PCC', 'MSE']])

In [6]:
results = {}

### +bilinear scenario

In [10]:
folder = 'bilinear_linear_models'

results[folder] = {}
for scene in scenes:
    # Create folders
    scene_folder = os.path.join(data_folder, folder, scene)
    result_folder = os.path.join(results_folder, folder, scene)

    # Generate data
    for seed in data_seeds:
        _ = generate_dataset.run(scene, num_endmembers, spectral_size, image_size=image_size, output_dir=os.path.join(scene_folder, f'{scene}{seed}'), noise=True, baseline=True, cosmic_spikes=True, realistic_endmembers=True, mixture_mode='bilinear', seed=seed)

    # Run experiment
    metrics = experiment.run(glob.glob(scene_folder + '/*'), num_endmembers + 1, model_replicates=model_replicates, epochs=epochs, experiment_folder=result_folder, seed=experiment_seed, ae_loss=ae_loss)

    results[folder][scene] = metrics

clear_output()

for scene, metrics in results[folder].items():
    print("Results for scene: ", scene)
    display_results(metrics)

Results for scene:  chessboard


,SAD,PCC,MSE
index,,,
PCA,0.967 ± 0.074,0.381 ± 0.093,88.132 ± 50.053
N-FINDR + FCLS,0.361 ± 0.062,0.19 ± 0.085,0.072 ± 0.04
VCA + FCLS,0.173 ± 0.079,0.091 ± 0.094,0.061 ± 0.038
Dense AE,0.045 ± 0.024,0.005 ± 0.01,0.03 ± 0.023
Convolutional AE,0.054 ± 0.013,0.003 ± 0.002,0.019 ± 0.008
Transformer AE,0.039 ± 0.005,0.001 ± 0.0,0.018 ± 0.009
Convolutional Transformer AE,0.04 ± 0.005,0.002 ± 0.001,0.018 ± 0.008


Results for scene:  gaussian


,SAD,PCC,MSE
index,,,
PCA,1.117 ± 0.071,0.543 ± 0.076,67.189 ± 48.404
N-FINDR + FCLS,0.456 ± 0.049,0.313 ± 0.065,0.039 ± 0.016
VCA + FCLS,0.391 ± 0.058,0.244 ± 0.085,0.057 ± 0.018
Dense AE,0.261 ± 0.072,0.111 ± 0.079,0.018 ± 0.013
Convolutional AE,0.209 ± 0.039,0.062 ± 0.026,0.009 ± 0.004
Transformer AE,0.204 ± 0.044,0.06 ± 0.034,0.011 ± 0.005
Convolutional Transformer AE,0.205 ± 0.055,0.065 ± 0.052,0.01 ± 0.005


Results for scene:  dirichlet


,SAD,PCC,MSE
index,,,
PCA,1.038 ± 0.112,0.444 ± 0.112,63.915 ± 48.423
N-FINDR + FCLS,0.287 ± 0.074,0.143 ± 0.083,0.025 ± 0.015
VCA + FCLS,0.277 ± 0.082,0.14 ± 0.1,0.03 ± 0.015
Dense AE,0.099 ± 0.016,0.008 ± 0.007,0.01 ± 0.004
Convolutional AE,0.093 ± 0.011,0.007 ± 0.001,0.008 ± 0.003
Transformer AE,0.107 ± 0.012,0.007 ± 0.001,0.008 ± 0.003
Convolutional Transformer AE,0.111 ± 0.031,0.017 ± 0.047,0.008 ± 0.003


##### +bilinear scenario - bilinear models

In [11]:
scenes = ['gaussian', 'dirichlet'] # no bilinear mixtures in chessboard

folder = 'bilinear_bilinear_models'

results[folder] = {}
for scene in scenes:
    # Create folders
    scene_folder = os.path.join(data_folder, folder, scene)
    result_folder = os.path.join(results_folder, folder, scene)

    # Generate data
    for seed in data_seeds:
        _ = generate_dataset.run(scene, num_endmembers, spectral_size, image_size=image_size, output_dir=os.path.join(scene_folder, f'{scene}{seed}'), noise=True, baseline=True, cosmic_spikes=True, realistic_endmembers=True, mixture_mode='bilinear', seed=seed)

    # Run experiment
    metrics = experiment.run(glob.glob(scene_folder + '/*'), num_endmembers + 1, model_replicates=model_replicates, decoder_type='fanbilinear', epochs=epochs, experiment_folder=result_folder, seed=experiment_seed, include_conventional=False, ae_loss=ae_loss)

    results[folder][scene] = metrics

clear_output()

for scene, metrics in results[folder].items():
    print("Results for scene: ", scene)
    display_results(metrics)

Results for scene:  gaussian


,SAD,PCC,MSE
index,,,
PCA,NaN,NaN,NaN
N-FINDR + FCLS,NaN,NaN,NaN
VCA + FCLS,NaN,NaN,NaN
Dense AE,0.247 ± 0.07,0.107 ± 0.086,0.017 ± 0.011
Convolutional AE,0.194 ± 0.03,0.053 ± 0.019,0.01 ± 0.004
Transformer AE,0.222 ± 0.084,0.091 ± 0.095,0.012 ± 0.008
Convolutional Transformer AE,0.208 ± 0.051,0.066 ± 0.045,0.011 ± 0.006


Results for scene:  dirichlet


,SAD,PCC,MSE
index,,,
PCA,NaN,NaN,NaN
N-FINDR + FCLS,NaN,NaN,NaN
VCA + FCLS,NaN,NaN,NaN
Dense AE,0.094 ± 0.011,0.006 ± 0.002,0.01 ± 0.005
Convolutional AE,0.087 ± 0.01,0.006 ± 0.001,0.008 ± 0.003
Transformer AE,0.105 ± 0.012,0.007 ± 0.001,0.008 ± 0.003
Convolutional Transformer AE,0.1 ± 0.01,0.007 ± 0.002,0.008 ± 0.003
